<h1>Life Cycle Scenario Analysis of Chinese-built High-speed Rail in Continental Southeast Asia</h1>
<h2>Model Implementation and Scenario Analysis</h2>

The present Jupyter notebook presents the code used to implement the life cycle model, apply it to the selected scenarios from the perspective of each considered country, and collect and export the results.

The notebook can be run as-is, assuming that one has access to the required packages, listed in the first code cell, and has downloaded the required data, which are saved in the model repository along with the notebook and model code. Excepting the packages numpy, pandas, and os, nothing outside of the repository is required for operation.

Running the full notebook will export selected calculations to csv files in the specified export directory, export_dir, listed in the second code cell. All model assumptions can be adjusted in the third code cell.

For further information on the model calculations, see the model.py and calc.py module documentation by entering help(model) and help(calc), respectively, after they have been imported. For complete details, each module includes full, PEP 8 compliant comments.

# Model Setup

## Settings and Imports

In [1]:
# Import required packages
import hsrlca.model as model
import hsrlca.calc as calc
import numpy as np  # model implemented with version 1.16.5
import pandas as pd  # model implemented with version 0.25.1
import os.path

In [2]:
# Set results export directory
export_dir = 'results'

## Assumptions

In [3]:
# Functional Unit Normalization

## Average number of seats per train
avg_train_capacity = 1000
## Average percentage of seats filled per train (between 0 and 1)
avg_train_capacity_filled = 0.70
## Average number of daily trips per train
avg_daily_trips_per_train = 2
## Average train trip distance in km
avg_train_trip_distance = 450
## Average train lifespan in years
avg_train_lifespan = 20
## Average number of trains running across all home_country tracks
avg_number_active_trains = 135
## Average train mass
avg_train_mass = 409000
## Average infrastructure lifespan in years
avg_infrastructure_lifespan = 100
## Average percent of all track that is ballasted
avg_pct_ballasted_track = 0.30
## Number of days in a year
days = 365.25

# Transportation

## Specify estimated proportion of goods transported by freight rail (rest allocated to freight lorry)
rail_allocation = 0.5

# Data

## Base Data

In [4]:
# Data directory name
data_dir = 'data'

# General data
## Unit process requirements, per unit output, as a square matrix
up_inputs_base = 'unit_process_inputs_master.csv'
## Unit process emissions, per unit output
up_emissions_base = 'unit_process_emissions_base.csv'
## Average distances implied by each possible international trade route for individual components
trade_distances = 'trade_distances.csv'
## National energy supply data
national_energy_supply = 'national_energy_supply.csv'
## Energy emissions per unit output (kWh)
unit_energy_emissions = 'unit_energy_emissions.csv'
## Conversion factors yielding CO2eq and SO2eq, respectively
emissions_eq_conversion = 'emissions_eq_conversions.csv'

## Scenario Data

In [5]:
# Differentiated scenario data
## Schedules of countries in which each unit process occurs

### Autarky scenario (no trade, all domestic production)
up_countries_CHN_aut = 'up_countries_CHN_aut.csv' # China under autarky
up_countries_KHM_aut = 'up_countries_KHM_aut.csv' # Cambodia under autarky
up_countries_LAO_aut = 'up_countries_LAO_aut.csv' # Lao PDR under autarky
up_countries_MMR_aut = 'up_countries_MMR_aut.csv' # Myanmar under autarky
up_countries_THA_aut = 'up_countries_THA_aut.csv' # Thailand under autarky
up_countries_VNM_aut = 'up_countries_VNM_aut.csv' # Vietnam under autarky

### Trade scenario 1: Furthest transportation for all inputs (transport worst case)
up_countries_CHN_far = 'up_countries_CHN_far.csv' # China under trade scenario 1
up_countries_KHM_far = 'up_countries_KHM_far.csv' # Cambodia under trade scenario 1
up_countries_LAO_far = 'up_countries_LAO_far.csv' # Lao PDR under trade scenario 1
up_countries_MMR_far = 'up_countries_MMR_far.csv' # Myanmar under trade scenario 1
up_countries_THA_far = 'up_countries_THA_far.csv' # Thailand under trade scenario 1
up_countries_VNM_far = 'up_countries_VNM_far.csv' # Vietnam under trade scenario 1

### Trade scenario 2: Closest (non-domestic) transportation for all inputs (transport best case)
up_countries_CHN_close = 'up_countries_CHN_close.csv' # China under trade scenario 2
up_countries_KHM_close = 'up_countries_KHM_close.csv' # Cambodia under trade scenario 2
up_countries_LAO_close = 'up_countries_LAO_close.csv' # Lao PDR under trade scenario 2
up_countries_MMR_close = 'up_countries_MMR_close.csv' # Myanmar under trade scenario 2
up_countries_THA_close = 'up_countries_THA_close.csv' # Thailand under trade scenario 2
up_countries_VNM_close = 'up_countries_VNM_close.csv' # Vietnam under trade scenario 2

### Trade scenario 3: Most carbon-intensive energy mix for all inputs (emissions worst case)
up_countries_CHN_maxemiss = 'up_countries_CHN_maxemiss.csv' # China under trade scenario 3
up_countries_KHM_maxemiss = 'up_countries_KHM_maxemiss.csv' # Cambodia under trade scenario 3
up_countries_LAO_maxemiss = 'up_countries_LAO_maxemiss.csv' # Lao PDR under trade scenario 3
up_countries_MMR_maxemiss = 'up_countries_MMR_maxemiss.csv' # Myanmar under trade scenario 3
up_countries_THA_maxemiss = 'up_countries_THA_maxemiss.csv' # Thailand under trade scenario 3
up_countries_VNM_maxemiss = 'up_countries_VNM_maxemiss.csv' # Vietnam under trade scenario 3

### Trade scenario 4: Least carbon-intensive (non-domestic) energy mix for all inputs (emissions best case)
up_countries_CHN_minemiss = 'up_countries_CHN_minemiss.csv' # China under trade scenario 4
up_countries_KHM_minemiss = 'up_countries_KHM_minemiss.csv' # Cambodia under trade scenario 4
up_countries_LAO_minemiss = 'up_countries_LAO_minemiss.csv' # Lao PDR under trade scenario 4
up_countries_MMR_minemiss = 'up_countries_MMR_minemiss.csv' # Myanmar under trade scenario 4
up_countries_THA_minemiss = 'up_countries_THA_minemiss.csv' # Thailand under trade scenario 4
up_countries_VNM_minemiss = 'up_countries_VNM_minemiss.csv' # Vietnam under trade scenario 4

# Modeling

## China Models

### Scenario 1: Autarky

In [6]:
CHN_aut = model.Scenario('China')

CHN_aut.set_data(data_dir,
                 up_inputs_base,
                 up_emissions_base,
                 national_energy_supply,
                 unit_energy_emissions,
                 emissions_eq_conversion,
                 trade_distances,
                 rail_allocation)

CHN_aut.set_scenario(up_countries_CHN_aut)

CHN_aut.set_norm_params(avg_train_capacity,
                        avg_train_capacity_filled,
                        avg_daily_trips_per_train,
                        avg_train_trip_distance,
                        avg_train_lifespan,
                        avg_number_active_trains,
                        avg_train_mass,
                        avg_infrastructure_lifespan,
                        avg_pct_ballasted_track,
                        days)

CHN_aut.make_trade_schedule()
CHN_aut.make_transport_schedule()
CHN_aut.update_inputs_transportation()
CHN_aut.get_energy_mix()
CHN_aut.get_energy_mix_emissions()
CHN_aut.update_emissions()
CHN_aut.update_inputs_electricity()
CHN_aut.update_inputs_passengers()
CHN_aut.make_emissions_schedule()
CHN_aut.get_total_requirements()
CHN_aut.get_total_emissions()
CHN_aut.get_phase_results()
CHN_aut.get_condensed_results()
CHN_aut.get_total_phase_impacts()
CHN_aut.get_total_lifetime_impacts()

C:\Users\Cory\Python\HSR-LCA\HSRLCA\hsrlca\calc.py:171: FutureWarning: The 'get_values' method is deprecated and will be removed in a future version. Use '.values' or 'np.asarray(..)' instead.
  up_emissions_complete.loc['electricity_generation'] = energy_mix_emissions.get_values()


### Scenario 2: All Trade via Longest Trade Route

In [7]:
CHN_far = model.Scenario('China')

CHN_far.set_data(data_dir,
                 up_inputs_base,
                 up_emissions_base,
                 national_energy_supply,
                 unit_energy_emissions,
                 emissions_eq_conversion,
                 trade_distances,
                 rail_allocation)

CHN_far.set_scenario(up_countries_CHN_far)

CHN_far.set_norm_params(avg_train_capacity,
                        avg_train_capacity_filled,
                        avg_daily_trips_per_train,
                        avg_train_trip_distance,
                        avg_train_lifespan,
                        avg_number_active_trains,
                        avg_train_mass,
                        avg_infrastructure_lifespan,
                        avg_pct_ballasted_track,
                        days)

CHN_far.make_trade_schedule()
CHN_far.make_transport_schedule()
CHN_far.update_inputs_transportation()
CHN_far.get_energy_mix()
CHN_far.get_energy_mix_emissions()
CHN_far.update_emissions()
CHN_far.update_inputs_electricity()
CHN_far.update_inputs_passengers()
CHN_far.make_emissions_schedule()
CHN_far.get_total_requirements()
CHN_far.get_total_emissions()
CHN_far.get_phase_results()
CHN_far.get_condensed_results()
CHN_far.get_total_phase_impacts()
CHN_far.get_total_lifetime_impacts()

### Scenario 3: All Trade via Shortest Trade Route

In [8]:
CHN_close = model.Scenario('China')

CHN_close.set_data(data_dir,
                 up_inputs_base,
                 up_emissions_base,
                 national_energy_supply,
                 unit_energy_emissions,
                 emissions_eq_conversion,
                 trade_distances,
                 rail_allocation)

CHN_close.set_scenario(up_countries_CHN_close)

CHN_close.set_norm_params(avg_train_capacity,
                        avg_train_capacity_filled,
                        avg_daily_trips_per_train,
                        avg_train_trip_distance,
                        avg_train_lifespan,
                        avg_number_active_trains,
                        avg_train_mass,
                        avg_infrastructure_lifespan,
                        avg_pct_ballasted_track,
                        days)

CHN_close.make_trade_schedule()
CHN_close.make_transport_schedule()
CHN_close.update_inputs_transportation()
CHN_close.get_energy_mix()
CHN_close.get_energy_mix_emissions()
CHN_close.update_emissions()
CHN_close.update_inputs_electricity()
CHN_close.update_inputs_passengers()
CHN_close.make_emissions_schedule()
CHN_close.get_total_requirements()
CHN_close.get_total_emissions()
CHN_close.get_phase_results()
CHN_close.get_condensed_results()
CHN_close.get_total_phase_impacts()
CHN_close.get_total_lifetime_impacts()

### Scenario 4: All Trade with Country with Highest-Emission (CO2) Energy Mix

In [9]:
CHN_maxemiss = model.Scenario('China')

CHN_maxemiss.set_data(data_dir,
                 up_inputs_base,
                 up_emissions_base,
                 national_energy_supply,
                 unit_energy_emissions,
                 emissions_eq_conversion,
                 trade_distances,
                 rail_allocation)

CHN_maxemiss.set_scenario(up_countries_CHN_maxemiss)

CHN_maxemiss.set_norm_params(avg_train_capacity,
                        avg_train_capacity_filled,
                        avg_daily_trips_per_train,
                        avg_train_trip_distance,
                        avg_train_lifespan,
                        avg_number_active_trains,
                        avg_train_mass,
                        avg_infrastructure_lifespan,
                        avg_pct_ballasted_track,
                        days)

CHN_maxemiss.make_trade_schedule()
CHN_maxemiss.make_transport_schedule()
CHN_maxemiss.update_inputs_transportation()
CHN_maxemiss.get_energy_mix()
CHN_maxemiss.get_energy_mix_emissions()
CHN_maxemiss.update_emissions()
CHN_maxemiss.update_inputs_electricity()
CHN_maxemiss.update_inputs_passengers()
CHN_maxemiss.make_emissions_schedule()
CHN_maxemiss.get_total_requirements()
CHN_maxemiss.get_total_emissions()
CHN_maxemiss.get_phase_results()
CHN_maxemiss.get_condensed_results()
CHN_maxemiss.get_total_phase_impacts()
CHN_maxemiss.get_total_lifetime_impacts()

### Scenario 5: All Trade with Country with Lowest-Emission (CO2) Energy Mix

In [10]:
CHN_minemiss = model.Scenario('China')

CHN_minemiss.set_data(data_dir,
                 up_inputs_base,
                 up_emissions_base,
                 national_energy_supply,
                 unit_energy_emissions,
                 emissions_eq_conversion,
                 trade_distances,
                 rail_allocation)

CHN_minemiss.set_scenario(up_countries_CHN_minemiss)

CHN_minemiss.set_norm_params(avg_train_capacity,
                        avg_train_capacity_filled,
                        avg_daily_trips_per_train,
                        avg_train_trip_distance,
                        avg_train_lifespan,
                        avg_number_active_trains,
                        avg_train_mass,
                        avg_infrastructure_lifespan,
                        avg_pct_ballasted_track,
                        days)

CHN_minemiss.make_trade_schedule()
CHN_minemiss.make_transport_schedule()
CHN_minemiss.update_inputs_transportation()
CHN_minemiss.get_energy_mix()
CHN_minemiss.get_energy_mix_emissions()
CHN_minemiss.update_emissions()
CHN_minemiss.update_inputs_electricity()
CHN_minemiss.update_inputs_passengers()
CHN_minemiss.make_emissions_schedule()
CHN_minemiss.get_total_requirements()
CHN_minemiss.get_total_emissions()
CHN_minemiss.get_phase_results()
CHN_minemiss.get_condensed_results()
CHN_minemiss.get_total_phase_impacts()
CHN_minemiss.get_total_lifetime_impacts()

## Cambodia Models

### Scenario 1: Autarky

In [11]:
KHM_aut = model.Scenario('Cambodia', verbose=False)

KHM_aut.set_data(data_dir,
                 up_inputs_base,
                 up_emissions_base,
                 national_energy_supply,
                 unit_energy_emissions,
                 emissions_eq_conversion,
                 trade_distances,
                 rail_allocation)

KHM_aut.set_scenario(up_countries_KHM_aut)

KHM_aut.set_norm_params(avg_train_capacity,
                        avg_train_capacity_filled,
                        avg_daily_trips_per_train,
                        avg_train_trip_distance,
                        avg_train_lifespan,
                        avg_number_active_trains,
                        avg_train_mass,
                        avg_infrastructure_lifespan,
                        avg_pct_ballasted_track,
                        days)

KHM_aut.make_trade_schedule()
KHM_aut.make_transport_schedule()
KHM_aut.update_inputs_transportation()
KHM_aut.get_energy_mix()
KHM_aut.get_energy_mix_emissions()
KHM_aut.update_emissions()
KHM_aut.update_inputs_electricity()
KHM_aut.update_inputs_passengers()
KHM_aut.make_emissions_schedule()
KHM_aut.get_total_requirements()
KHM_aut.get_total_emissions()
KHM_aut.get_phase_results()
KHM_aut.get_condensed_results()
KHM_aut.get_total_phase_impacts()
KHM_aut.get_total_lifetime_impacts()

### Scenario 2: All Trade via Longest Trade Route

In [12]:
KHM_far = model.Scenario('Cambodia', verbose=False)

KHM_far.set_data(data_dir,
                 up_inputs_base,
                 up_emissions_base,
                 national_energy_supply,
                 unit_energy_emissions,
                 emissions_eq_conversion,
                 trade_distances,
                 rail_allocation)

KHM_far.set_scenario(up_countries_KHM_far)

KHM_far.set_norm_params(avg_train_capacity,
                        avg_train_capacity_filled,
                        avg_daily_trips_per_train,
                        avg_train_trip_distance,
                        avg_train_lifespan,
                        avg_number_active_trains,
                        avg_train_mass,
                        avg_infrastructure_lifespan,
                        avg_pct_ballasted_track,
                        days)

KHM_far.make_trade_schedule()
KHM_far.make_transport_schedule()
KHM_far.update_inputs_transportation()
KHM_far.get_energy_mix()
KHM_far.get_energy_mix_emissions()
KHM_far.update_emissions()
KHM_far.update_inputs_electricity()
KHM_far.update_inputs_passengers()
KHM_far.make_emissions_schedule()
KHM_far.get_total_requirements()
KHM_far.get_total_emissions()
KHM_far.get_phase_results()
KHM_far.get_condensed_results()
KHM_far.get_total_phase_impacts()
KHM_far.get_total_lifetime_impacts()

### Scenario 3: All Trade via Shortest Trade Route

In [13]:
KHM_close = model.Scenario('Cambodia', verbose=False)

KHM_close.set_data(data_dir,
                 up_inputs_base,
                 up_emissions_base,
                 national_energy_supply,
                 unit_energy_emissions,
                 emissions_eq_conversion,
                 trade_distances,
                 rail_allocation)

KHM_close.set_scenario(up_countries_KHM_close)

KHM_close.set_norm_params(avg_train_capacity,
                        avg_train_capacity_filled,
                        avg_daily_trips_per_train,
                        avg_train_trip_distance,
                        avg_train_lifespan,
                        avg_number_active_trains,
                        avg_train_mass,
                        avg_infrastructure_lifespan,
                        avg_pct_ballasted_track,
                        days)

KHM_close.make_trade_schedule()
KHM_close.make_transport_schedule()
KHM_close.update_inputs_transportation()
KHM_close.get_energy_mix()
KHM_close.get_energy_mix_emissions()
KHM_close.update_emissions()
KHM_close.update_inputs_electricity()
KHM_close.update_inputs_passengers()
KHM_close.make_emissions_schedule()
KHM_close.get_total_requirements()
KHM_close.get_total_emissions()
KHM_close.get_phase_results()
KHM_close.get_condensed_results()
KHM_close.get_total_phase_impacts()
KHM_close.get_total_lifetime_impacts()

### Scenario 4: All Trade with Country with Highest-Emission (CO2) Energy Mix

In [14]:
KHM_maxemiss = model.Scenario('Cambodia', verbose=False)

KHM_maxemiss.set_data(data_dir,
                 up_inputs_base,
                 up_emissions_base,
                 national_energy_supply,
                 unit_energy_emissions,
                 emissions_eq_conversion,
                 trade_distances,
                 rail_allocation)

KHM_maxemiss.set_scenario(up_countries_KHM_maxemiss)

KHM_maxemiss.set_norm_params(avg_train_capacity,
                        avg_train_capacity_filled,
                        avg_daily_trips_per_train,
                        avg_train_trip_distance,
                        avg_train_lifespan,
                        avg_number_active_trains,
                        avg_train_mass,
                        avg_infrastructure_lifespan,
                        avg_pct_ballasted_track,
                        days)

KHM_maxemiss.make_trade_schedule()
KHM_maxemiss.make_transport_schedule()
KHM_maxemiss.update_inputs_transportation()
KHM_maxemiss.get_energy_mix()
KHM_maxemiss.get_energy_mix_emissions()
KHM_maxemiss.update_emissions()
KHM_maxemiss.update_inputs_electricity()
KHM_maxemiss.update_inputs_passengers()
KHM_maxemiss.make_emissions_schedule()
KHM_maxemiss.get_total_requirements()
KHM_maxemiss.get_total_emissions()
KHM_maxemiss.get_phase_results()
KHM_maxemiss.get_condensed_results()
KHM_maxemiss.get_total_phase_impacts()
KHM_maxemiss.get_total_lifetime_impacts()

### Scenario 5: All Trade with Country with Lowest-Emission (CO2) Energy Mix

In [15]:
KHM_minemiss = model.Scenario('Cambodia', verbose=False)

KHM_minemiss.set_data(data_dir,
                 up_inputs_base,
                 up_emissions_base,
                 national_energy_supply,
                 unit_energy_emissions,
                 emissions_eq_conversion,
                 trade_distances,
                 rail_allocation)

KHM_minemiss.set_scenario(up_countries_KHM_minemiss)

KHM_minemiss.set_norm_params(avg_train_capacity,
                        avg_train_capacity_filled,
                        avg_daily_trips_per_train,
                        avg_train_trip_distance,
                        avg_train_lifespan,
                        avg_number_active_trains,
                        avg_train_mass,
                        avg_infrastructure_lifespan,
                        avg_pct_ballasted_track,
                        days)

KHM_minemiss.make_trade_schedule()
KHM_minemiss.make_transport_schedule()
KHM_minemiss.update_inputs_transportation()
KHM_minemiss.get_energy_mix()
KHM_minemiss.get_energy_mix_emissions()
KHM_minemiss.update_emissions()
KHM_minemiss.update_inputs_electricity()
KHM_minemiss.update_inputs_passengers()
KHM_minemiss.make_emissions_schedule()
KHM_minemiss.get_total_requirements()
KHM_minemiss.get_total_emissions()
KHM_minemiss.get_phase_results()
KHM_minemiss.get_condensed_results()
KHM_minemiss.get_total_phase_impacts()
KHM_minemiss.get_total_lifetime_impacts()

## Lao PDR Models

### Scenario 1: Autarky

In [16]:
LAO_aut = model.Scenario('LaoPDR')

LAO_aut.set_data(data_dir,
                 up_inputs_base,
                 up_emissions_base,
                 national_energy_supply,
                 unit_energy_emissions,
                 emissions_eq_conversion,
                 trade_distances,
                 rail_allocation)

LAO_aut.set_scenario(up_countries_LAO_aut)

LAO_aut.set_norm_params(avg_train_capacity,
                        avg_train_capacity_filled,
                        avg_daily_trips_per_train,
                        avg_train_trip_distance,
                        avg_train_lifespan,
                        avg_number_active_trains,
                        avg_train_mass,
                        avg_infrastructure_lifespan,
                        avg_pct_ballasted_track,
                        days)

LAO_aut.make_trade_schedule()
LAO_aut.make_transport_schedule()
LAO_aut.update_inputs_transportation()
LAO_aut.get_energy_mix()
LAO_aut.get_energy_mix_emissions()
LAO_aut.update_emissions()
LAO_aut.update_inputs_electricity()
LAO_aut.update_inputs_passengers()
LAO_aut.make_emissions_schedule()
LAO_aut.get_total_requirements()
LAO_aut.get_total_emissions()
LAO_aut.get_phase_results()
LAO_aut.get_condensed_results()
LAO_aut.get_total_phase_impacts()
LAO_aut.get_total_lifetime_impacts()

### Scenario 2: All Trade via Longest Trade Route

In [17]:
LAO_far = model.Scenario('LaoPDR')

LAO_far.set_data(data_dir,
                 up_inputs_base,
                 up_emissions_base,
                 national_energy_supply,
                 unit_energy_emissions,
                 emissions_eq_conversion,
                 trade_distances,
                 rail_allocation)

LAO_far.set_scenario(up_countries_LAO_far)

LAO_far.set_norm_params(avg_train_capacity,
                        avg_train_capacity_filled,
                        avg_daily_trips_per_train,
                        avg_train_trip_distance,
                        avg_train_lifespan,
                        avg_number_active_trains,
                        avg_train_mass,
                        avg_infrastructure_lifespan,
                        avg_pct_ballasted_track,
                        days)

LAO_far.make_trade_schedule()
LAO_far.make_transport_schedule()
LAO_far.update_inputs_transportation()
LAO_far.get_energy_mix()
LAO_far.get_energy_mix_emissions()
LAO_far.update_emissions()
LAO_far.update_inputs_electricity()
LAO_far.update_inputs_passengers()
LAO_far.make_emissions_schedule()
LAO_far.get_total_requirements()
LAO_far.get_total_emissions()
LAO_far.get_phase_results()
LAO_far.get_condensed_results()
LAO_far.get_total_phase_impacts()
LAO_far.get_total_lifetime_impacts()

### Scenario 3: All Trade via Shortest Trade Route

In [18]:
LAO_close = model.Scenario('LaoPDR')

LAO_close.set_data(data_dir,
                 up_inputs_base,
                 up_emissions_base,
                 national_energy_supply,
                 unit_energy_emissions,
                 emissions_eq_conversion,
                 trade_distances,
                 rail_allocation)

LAO_close.set_scenario(up_countries_LAO_close)

LAO_close.set_norm_params(avg_train_capacity,
                        avg_train_capacity_filled,
                        avg_daily_trips_per_train,
                        avg_train_trip_distance,
                        avg_train_lifespan,
                        avg_number_active_trains,
                        avg_train_mass,
                        avg_infrastructure_lifespan,
                        avg_pct_ballasted_track,
                        days)

LAO_close.make_trade_schedule()
LAO_close.make_transport_schedule()
LAO_close.update_inputs_transportation()
LAO_close.get_energy_mix()
LAO_close.get_energy_mix_emissions()
LAO_close.update_emissions()
LAO_close.update_inputs_electricity()
LAO_close.update_inputs_passengers()
LAO_close.make_emissions_schedule()
LAO_close.get_total_requirements()
LAO_close.get_total_emissions()
LAO_close.get_phase_results()
LAO_close.get_condensed_results()
LAO_close.get_total_phase_impacts()
LAO_close.get_total_lifetime_impacts()

### Scenario 4: All Trade with Country with Highest-Emission (CO2) Energy Mix

In [19]:
LAO_maxemiss = model.Scenario('LaoPDR')

LAO_maxemiss.set_data(data_dir,
                 up_inputs_base,
                 up_emissions_base,
                 national_energy_supply,
                 unit_energy_emissions,
                 emissions_eq_conversion,
                 trade_distances,
                 rail_allocation)

LAO_maxemiss.set_scenario(up_countries_LAO_maxemiss)

LAO_maxemiss.set_norm_params(avg_train_capacity,
                        avg_train_capacity_filled,
                        avg_daily_trips_per_train,
                        avg_train_trip_distance,
                        avg_train_lifespan,
                        avg_number_active_trains,
                        avg_train_mass,
                        avg_infrastructure_lifespan,
                        avg_pct_ballasted_track,
                        days)

LAO_maxemiss.make_trade_schedule()
LAO_maxemiss.make_transport_schedule()
LAO_maxemiss.update_inputs_transportation()
LAO_maxemiss.get_energy_mix()
LAO_maxemiss.get_energy_mix_emissions()
LAO_maxemiss.update_emissions()
LAO_maxemiss.update_inputs_electricity()
LAO_maxemiss.update_inputs_passengers()
LAO_maxemiss.make_emissions_schedule()
LAO_maxemiss.get_total_requirements()
LAO_maxemiss.get_total_emissions()
LAO_maxemiss.get_phase_results()
LAO_maxemiss.get_condensed_results()
LAO_maxemiss.get_total_phase_impacts()
LAO_maxemiss.get_total_lifetime_impacts()

### Scenario 5: All Trade with Country with Lowest-Emission (CO2) Energy Mix

In [20]:
LAO_minemiss = model.Scenario('LaoPDR')

LAO_minemiss.set_data(data_dir,
                 up_inputs_base,
                 up_emissions_base,
                 national_energy_supply,
                 unit_energy_emissions,
                 emissions_eq_conversion,
                 trade_distances,
                 rail_allocation)

LAO_minemiss.set_scenario(up_countries_LAO_minemiss)

LAO_minemiss.set_norm_params(avg_train_capacity,
                        avg_train_capacity_filled,
                        avg_daily_trips_per_train,
                        avg_train_trip_distance,
                        avg_train_lifespan,
                        avg_number_active_trains,
                        avg_train_mass,
                        avg_infrastructure_lifespan,
                        avg_pct_ballasted_track,
                        days)

LAO_minemiss.make_trade_schedule()
LAO_minemiss.make_transport_schedule()
LAO_minemiss.update_inputs_transportation()
LAO_minemiss.get_energy_mix()
LAO_minemiss.get_energy_mix_emissions()
LAO_minemiss.update_emissions()
LAO_minemiss.update_inputs_electricity()
LAO_minemiss.update_inputs_passengers()
LAO_minemiss.make_emissions_schedule()
LAO_minemiss.get_total_requirements()
LAO_minemiss.get_total_emissions()
LAO_minemiss.get_phase_results()
LAO_minemiss.get_condensed_results()
LAO_minemiss.get_total_phase_impacts()
LAO_minemiss.get_total_lifetime_impacts()

## Myanmar Models

### Scenario 1: Autarky

In [21]:
MMR_aut = model.Scenario('Myanmar')

MMR_aut.set_data(data_dir,
                 up_inputs_base,
                 up_emissions_base,
                 national_energy_supply,
                 unit_energy_emissions,
                 emissions_eq_conversion,
                 trade_distances,
                 rail_allocation)

MMR_aut.set_scenario(up_countries_MMR_aut)

MMR_aut.set_norm_params(avg_train_capacity,
                        avg_train_capacity_filled,
                        avg_daily_trips_per_train,
                        avg_train_trip_distance,
                        avg_train_lifespan,
                        avg_number_active_trains,
                        avg_train_mass,
                        avg_infrastructure_lifespan,
                        avg_pct_ballasted_track,
                        days)

MMR_aut.make_trade_schedule()
MMR_aut.make_transport_schedule()
MMR_aut.update_inputs_transportation()
MMR_aut.get_energy_mix()
MMR_aut.get_energy_mix_emissions()
MMR_aut.update_emissions()
MMR_aut.update_inputs_electricity()
MMR_aut.update_inputs_passengers()
MMR_aut.make_emissions_schedule()
MMR_aut.get_total_requirements()
MMR_aut.get_total_emissions()
MMR_aut.get_phase_results()
MMR_aut.get_condensed_results()
MMR_aut.get_total_phase_impacts()
MMR_aut.get_total_lifetime_impacts()

### Scenario 2: All Trade via Longest Trade Route

In [22]:
MMR_far = model.Scenario('Myanmar')

MMR_far.set_data(data_dir,
                 up_inputs_base,
                 up_emissions_base,
                 national_energy_supply,
                 unit_energy_emissions,
                 emissions_eq_conversion,
                 trade_distances,
                 rail_allocation)

MMR_far.set_scenario(up_countries_MMR_far)

MMR_far.set_norm_params(avg_train_capacity,
                        avg_train_capacity_filled,
                        avg_daily_trips_per_train,
                        avg_train_trip_distance,
                        avg_train_lifespan,
                        avg_number_active_trains,
                        avg_train_mass,
                        avg_infrastructure_lifespan,
                        avg_pct_ballasted_track,
                        days)

MMR_far.make_trade_schedule()
MMR_far.make_transport_schedule()
MMR_far.update_inputs_transportation()
MMR_far.get_energy_mix()
MMR_far.get_energy_mix_emissions()
MMR_far.update_emissions()
MMR_far.update_inputs_electricity()
MMR_far.update_inputs_passengers()
MMR_far.make_emissions_schedule()
MMR_far.get_total_requirements()
MMR_far.get_total_emissions()
MMR_far.get_phase_results()
MMR_far.get_condensed_results()
MMR_far.get_total_phase_impacts()
MMR_far.get_total_lifetime_impacts()

### Scenario 3: All Trade via Shortest Trade Route

In [23]:
MMR_close = model.Scenario('Myanmar')

MMR_close.set_data(data_dir,
                 up_inputs_base,
                 up_emissions_base,
                 national_energy_supply,
                 unit_energy_emissions,
                 emissions_eq_conversion,
                 trade_distances,
                 rail_allocation)

MMR_close.set_scenario(up_countries_MMR_close)

MMR_close.set_norm_params(avg_train_capacity,
                        avg_train_capacity_filled,
                        avg_daily_trips_per_train,
                        avg_train_trip_distance,
                        avg_train_lifespan,
                        avg_number_active_trains,
                        avg_train_mass,
                        avg_infrastructure_lifespan,
                        avg_pct_ballasted_track,
                        days)

MMR_close.make_trade_schedule()
MMR_close.make_transport_schedule()
MMR_close.update_inputs_transportation()
MMR_close.get_energy_mix()
MMR_close.get_energy_mix_emissions()
MMR_close.update_emissions()
MMR_close.update_inputs_electricity()
MMR_close.update_inputs_passengers()
MMR_close.make_emissions_schedule()
MMR_close.get_total_requirements()
MMR_close.get_total_emissions()
MMR_close.get_phase_results()
MMR_close.get_condensed_results()
MMR_close.get_total_phase_impacts()
MMR_close.get_total_lifetime_impacts()

### Scenario 4: All Trade with Country with Highest-Emission (CO2) Energy Mix

In [24]:
MMR_maxemiss = model.Scenario('Myanmar')

MMR_maxemiss.set_data(data_dir,
                 up_inputs_base,
                 up_emissions_base,
                 national_energy_supply,
                 unit_energy_emissions,
                 emissions_eq_conversion,
                 trade_distances,
                 rail_allocation)

MMR_maxemiss.set_scenario(up_countries_MMR_maxemiss)

MMR_maxemiss.set_norm_params(avg_train_capacity,
                        avg_train_capacity_filled,
                        avg_daily_trips_per_train,
                        avg_train_trip_distance,
                        avg_train_lifespan,
                        avg_number_active_trains,
                        avg_train_mass,
                        avg_infrastructure_lifespan,
                        avg_pct_ballasted_track,
                        days)

MMR_maxemiss.make_trade_schedule()
MMR_maxemiss.make_transport_schedule()
MMR_maxemiss.update_inputs_transportation()
MMR_maxemiss.get_energy_mix()
MMR_maxemiss.get_energy_mix_emissions()
MMR_maxemiss.update_emissions()
MMR_maxemiss.update_inputs_electricity()
MMR_maxemiss.update_inputs_passengers()
MMR_maxemiss.make_emissions_schedule()
MMR_maxemiss.get_total_requirements()
MMR_maxemiss.get_total_emissions()
MMR_maxemiss.get_phase_results()
MMR_maxemiss.get_condensed_results()
MMR_maxemiss.get_total_phase_impacts()
MMR_maxemiss.get_total_lifetime_impacts()

### Scenario 5: All Trade with Country with Lowest-Emission (CO2) Energy Mix

In [25]:
MMR_minemiss = model.Scenario('Myanmar')

MMR_minemiss.set_data(data_dir,
                 up_inputs_base,
                 up_emissions_base,
                 national_energy_supply,
                 unit_energy_emissions,
                 emissions_eq_conversion,
                 trade_distances,
                 rail_allocation)

MMR_minemiss.set_scenario(up_countries_MMR_minemiss)

MMR_minemiss.set_norm_params(avg_train_capacity,
                        avg_train_capacity_filled,
                        avg_daily_trips_per_train,
                        avg_train_trip_distance,
                        avg_train_lifespan,
                        avg_number_active_trains,
                        avg_train_mass,
                        avg_infrastructure_lifespan,
                        avg_pct_ballasted_track,
                        days)

MMR_minemiss.make_trade_schedule()
MMR_minemiss.make_transport_schedule()
MMR_minemiss.update_inputs_transportation()
MMR_minemiss.get_energy_mix()
MMR_minemiss.get_energy_mix_emissions()
MMR_minemiss.update_emissions()
MMR_minemiss.update_inputs_electricity()
MMR_minemiss.update_inputs_passengers()
MMR_minemiss.make_emissions_schedule()
MMR_minemiss.get_total_requirements()
MMR_minemiss.get_total_emissions()
MMR_minemiss.get_phase_results()
MMR_minemiss.get_condensed_results()
MMR_minemiss.get_total_phase_impacts()
MMR_minemiss.get_total_lifetime_impacts()

## Thailand Models

### Scenario 1: Autarky

In [26]:
THA_aut = model.Scenario('Thailand')

THA_aut.set_data(data_dir,
                 up_inputs_base,
                 up_emissions_base,
                 national_energy_supply,
                 unit_energy_emissions,
                 emissions_eq_conversion,
                 trade_distances,
                 rail_allocation)

THA_aut.set_scenario(up_countries_THA_aut)

THA_aut.set_norm_params(avg_train_capacity,
                        avg_train_capacity_filled,
                        avg_daily_trips_per_train,
                        avg_train_trip_distance,
                        avg_train_lifespan,
                        avg_number_active_trains,
                        avg_train_mass,
                        avg_infrastructure_lifespan,
                        avg_pct_ballasted_track,
                        days)

THA_aut.make_trade_schedule()
THA_aut.make_transport_schedule()
THA_aut.update_inputs_transportation()
THA_aut.get_energy_mix()
THA_aut.get_energy_mix_emissions()
THA_aut.update_emissions()
THA_aut.update_inputs_electricity()
THA_aut.update_inputs_passengers()
THA_aut.make_emissions_schedule()
THA_aut.get_total_requirements()
THA_aut.get_total_emissions()
THA_aut.get_phase_results()
THA_aut.get_condensed_results()
THA_aut.get_total_phase_impacts()
THA_aut.get_total_lifetime_impacts()

### Scenario 2: All Trade via Longest Trade Route

In [27]:
THA_far = model.Scenario('Thailand')

THA_far.set_data(data_dir,
                 up_inputs_base,
                 up_emissions_base,
                 national_energy_supply,
                 unit_energy_emissions,
                 emissions_eq_conversion,
                 trade_distances,
                 rail_allocation)

THA_far.set_scenario(up_countries_THA_far)

THA_far.set_norm_params(avg_train_capacity,
                        avg_train_capacity_filled,
                        avg_daily_trips_per_train,
                        avg_train_trip_distance,
                        avg_train_lifespan,
                        avg_number_active_trains,
                        avg_train_mass,
                        avg_infrastructure_lifespan,
                        avg_pct_ballasted_track,
                        days)

THA_far.make_trade_schedule()
THA_far.make_transport_schedule()
THA_far.update_inputs_transportation()
THA_far.get_energy_mix()
THA_far.get_energy_mix_emissions()
THA_far.update_emissions()
THA_far.update_inputs_electricity()
THA_far.update_inputs_passengers()
THA_far.make_emissions_schedule()
THA_far.get_total_requirements()
THA_far.get_total_emissions()
THA_far.get_phase_results()
THA_far.get_condensed_results()
THA_far.get_total_phase_impacts()
THA_far.get_total_lifetime_impacts()

### Scenario 3: All Trade via Shortest Trade Route

In [28]:
THA_close = model.Scenario('Thailand')

THA_close.set_data(data_dir,
                 up_inputs_base,
                 up_emissions_base,
                 national_energy_supply,
                 unit_energy_emissions,
                 emissions_eq_conversion,
                 trade_distances,
                 rail_allocation)

THA_close.set_scenario(up_countries_THA_close)

THA_close.set_norm_params(avg_train_capacity,
                        avg_train_capacity_filled,
                        avg_daily_trips_per_train,
                        avg_train_trip_distance,
                        avg_train_lifespan,
                        avg_number_active_trains,
                        avg_train_mass,
                        avg_infrastructure_lifespan,
                        avg_pct_ballasted_track,
                        days)

THA_close.make_trade_schedule()
THA_close.make_transport_schedule()
THA_close.update_inputs_transportation()
THA_close.get_energy_mix()
THA_close.get_energy_mix_emissions()
THA_close.update_emissions()
THA_close.update_inputs_electricity()
THA_close.update_inputs_passengers()
THA_close.make_emissions_schedule()
THA_close.get_total_requirements()
THA_close.get_total_emissions()
THA_close.get_phase_results()
THA_close.get_condensed_results()
THA_close.get_total_phase_impacts()
THA_close.get_total_lifetime_impacts()

### Scenario 4: All Trade with Country with Highest-Emission (CO2) Energy Mix

In [29]:
THA_maxemiss = model.Scenario('Thailand')

THA_maxemiss.set_data(data_dir,
                 up_inputs_base,
                 up_emissions_base,
                 national_energy_supply,
                 unit_energy_emissions,
                 emissions_eq_conversion,
                 trade_distances,
                 rail_allocation)

THA_maxemiss.set_scenario(up_countries_THA_maxemiss)

THA_maxemiss.set_norm_params(avg_train_capacity,
                        avg_train_capacity_filled,
                        avg_daily_trips_per_train,
                        avg_train_trip_distance,
                        avg_train_lifespan,
                        avg_number_active_trains,
                        avg_train_mass,
                        avg_infrastructure_lifespan,
                        avg_pct_ballasted_track,
                        days)

THA_maxemiss.make_trade_schedule()
THA_maxemiss.make_transport_schedule()
THA_maxemiss.update_inputs_transportation()
THA_maxemiss.get_energy_mix()
THA_maxemiss.get_energy_mix_emissions()
THA_maxemiss.update_emissions()
THA_maxemiss.update_inputs_electricity()
THA_maxemiss.update_inputs_passengers()
THA_maxemiss.make_emissions_schedule()
THA_maxemiss.get_total_requirements()
THA_maxemiss.get_total_emissions()
THA_maxemiss.get_phase_results()
THA_maxemiss.get_condensed_results()
THA_maxemiss.get_total_phase_impacts()
THA_maxemiss.get_total_lifetime_impacts()

### Scenario 5: All Trade with Country with Lowest-Emission (CO2) Energy Mix

In [30]:
THA_minemiss = model.Scenario('Thailand')

THA_minemiss.set_data(data_dir,
                 up_inputs_base,
                 up_emissions_base,
                 national_energy_supply,
                 unit_energy_emissions,
                 emissions_eq_conversion,
                 trade_distances,
                 rail_allocation)

THA_minemiss.set_scenario(up_countries_THA_minemiss)

THA_minemiss.set_norm_params(avg_train_capacity,
                        avg_train_capacity_filled,
                        avg_daily_trips_per_train,
                        avg_train_trip_distance,
                        avg_train_lifespan,
                        avg_number_active_trains,
                        avg_train_mass,
                        avg_infrastructure_lifespan,
                        avg_pct_ballasted_track,
                        days)

THA_minemiss.make_trade_schedule()
THA_minemiss.make_transport_schedule()
THA_minemiss.update_inputs_transportation()
THA_minemiss.get_energy_mix()
THA_minemiss.get_energy_mix_emissions()
THA_minemiss.update_emissions()
THA_minemiss.update_inputs_electricity()
THA_minemiss.update_inputs_passengers()
THA_minemiss.make_emissions_schedule()
THA_minemiss.get_total_requirements()
THA_minemiss.get_total_emissions()
THA_minemiss.get_phase_results()
THA_minemiss.get_condensed_results()
THA_minemiss.get_total_phase_impacts()
THA_minemiss.get_total_lifetime_impacts()

## Vietnam Models

### Scenario 1: Autarky

In [31]:
VNM_aut = model.Scenario('Vietnam')

VNM_aut.set_data(data_dir,
                 up_inputs_base,
                 up_emissions_base,
                 national_energy_supply,
                 unit_energy_emissions,
                 emissions_eq_conversion,
                 trade_distances,
                 rail_allocation)

VNM_aut.set_scenario(up_countries_VNM_aut)

VNM_aut.set_norm_params(avg_train_capacity,
                        avg_train_capacity_filled,
                        avg_daily_trips_per_train,
                        avg_train_trip_distance,
                        avg_train_lifespan,
                        avg_number_active_trains,
                        avg_train_mass,
                        avg_infrastructure_lifespan,
                        avg_pct_ballasted_track,
                        days)

VNM_aut.make_trade_schedule()
VNM_aut.make_transport_schedule()
VNM_aut.update_inputs_transportation()
VNM_aut.get_energy_mix()
VNM_aut.get_energy_mix_emissions()
VNM_aut.update_emissions()
VNM_aut.update_inputs_electricity()
VNM_aut.update_inputs_passengers()
VNM_aut.make_emissions_schedule()
VNM_aut.get_total_requirements()
VNM_aut.get_total_emissions()
VNM_aut.get_phase_results()
VNM_aut.get_condensed_results()
VNM_aut.get_total_phase_impacts()
VNM_aut.get_total_lifetime_impacts()

### Scenario 2: All Trade via Longest Trade Route

In [32]:
VNM_far = model.Scenario('Vietnam')

VNM_far.set_data(data_dir,
                 up_inputs_base,
                 up_emissions_base,
                 national_energy_supply,
                 unit_energy_emissions,
                 emissions_eq_conversion,
                 trade_distances,
                 rail_allocation)

VNM_far.set_scenario(up_countries_VNM_far)

VNM_far.set_norm_params(avg_train_capacity,
                        avg_train_capacity_filled,
                        avg_daily_trips_per_train,
                        avg_train_trip_distance,
                        avg_train_lifespan,
                        avg_number_active_trains,
                        avg_train_mass,
                        avg_infrastructure_lifespan,
                        avg_pct_ballasted_track,
                        days)

VNM_far.make_trade_schedule()
VNM_far.make_transport_schedule()
VNM_far.update_inputs_transportation()
VNM_far.get_energy_mix()
VNM_far.get_energy_mix_emissions()
VNM_far.update_emissions()
VNM_far.update_inputs_electricity()
VNM_far.update_inputs_passengers()
VNM_far.make_emissions_schedule()
VNM_far.get_total_requirements()
VNM_far.get_total_emissions()
VNM_far.get_phase_results()
VNM_far.get_condensed_results()
VNM_far.get_total_phase_impacts()
VNM_far.get_total_lifetime_impacts()

### Scenario 3: All Trade via Shortest Trade Route

In [33]:
VNM_close = model.Scenario('Vietnam')

VNM_close.set_data(data_dir,
                 up_inputs_base,
                 up_emissions_base,
                 national_energy_supply,
                 unit_energy_emissions,
                 emissions_eq_conversion,
                 trade_distances,
                 rail_allocation)

VNM_close.set_scenario(up_countries_VNM_close)

VNM_close.set_norm_params(avg_train_capacity,
                        avg_train_capacity_filled,
                        avg_daily_trips_per_train,
                        avg_train_trip_distance,
                        avg_train_lifespan,
                        avg_number_active_trains,
                        avg_train_mass,
                        avg_infrastructure_lifespan,
                        avg_pct_ballasted_track,
                        days)

VNM_close.make_trade_schedule()
VNM_close.make_transport_schedule()
VNM_close.update_inputs_transportation()
VNM_close.get_energy_mix()
VNM_close.get_energy_mix_emissions()
VNM_close.update_emissions()
VNM_close.update_inputs_electricity()
VNM_close.update_inputs_passengers()
VNM_close.make_emissions_schedule()
VNM_close.get_total_requirements()
VNM_close.get_total_emissions()
VNM_close.get_phase_results()
VNM_close.get_condensed_results()
VNM_close.get_total_phase_impacts()
VNM_close.get_total_lifetime_impacts()

### Scenario 4: All Trade with Country with Highest-Emission (CO2) Energy Mix

In [34]:
VNM_maxemiss = model.Scenario('Vietnam')

VNM_maxemiss.set_data(data_dir,
                 up_inputs_base,
                 up_emissions_base,
                 national_energy_supply,
                 unit_energy_emissions,
                 emissions_eq_conversion,
                 trade_distances,
                 rail_allocation)

VNM_maxemiss.set_scenario(up_countries_VNM_maxemiss)

VNM_maxemiss.set_norm_params(avg_train_capacity,
                        avg_train_capacity_filled,
                        avg_daily_trips_per_train,
                        avg_train_trip_distance,
                        avg_train_lifespan,
                        avg_number_active_trains,
                        avg_train_mass,
                        avg_infrastructure_lifespan,
                        avg_pct_ballasted_track,
                        days)

VNM_maxemiss.make_trade_schedule()
VNM_maxemiss.make_transport_schedule()
VNM_maxemiss.update_inputs_transportation()
VNM_maxemiss.get_energy_mix()
VNM_maxemiss.get_energy_mix_emissions()
VNM_maxemiss.update_emissions()
VNM_maxemiss.update_inputs_electricity()
VNM_maxemiss.update_inputs_passengers()
VNM_maxemiss.make_emissions_schedule()
VNM_maxemiss.get_total_requirements()
VNM_maxemiss.get_total_emissions()
VNM_maxemiss.get_phase_results()
VNM_maxemiss.get_condensed_results()
VNM_maxemiss.get_total_phase_impacts()
VNM_maxemiss.get_total_lifetime_impacts()

### Scenario 5: All Trade with Country with Lowest-Emission (CO2) Energy Mix

In [35]:
VNM_minemiss = model.Scenario('Vietnam')

VNM_minemiss.set_data(data_dir,
                 up_inputs_base,
                 up_emissions_base,
                 national_energy_supply,
                 unit_energy_emissions,
                 emissions_eq_conversion,
                 trade_distances,
                 rail_allocation)

VNM_minemiss.set_scenario(up_countries_VNM_minemiss)

VNM_minemiss.set_norm_params(avg_train_capacity,
                        avg_train_capacity_filled,
                        avg_daily_trips_per_train,
                        avg_train_trip_distance,
                        avg_train_lifespan,
                        avg_number_active_trains,
                        avg_train_mass,
                        avg_infrastructure_lifespan,
                        avg_pct_ballasted_track,
                        days)

VNM_minemiss.make_trade_schedule()
VNM_minemiss.make_transport_schedule()
VNM_minemiss.update_inputs_transportation()
VNM_minemiss.get_energy_mix()
VNM_minemiss.get_energy_mix_emissions()
VNM_minemiss.update_emissions()
VNM_minemiss.update_inputs_electricity()
VNM_minemiss.update_inputs_passengers()
VNM_minemiss.make_emissions_schedule()
VNM_minemiss.get_total_requirements()
VNM_minemiss.get_total_emissions()
VNM_minemiss.get_phase_results()
VNM_minemiss.get_condensed_results()
VNM_minemiss.get_total_phase_impacts()
VNM_minemiss.get_total_lifetime_impacts()

## Display Impacts

In [36]:
# Control impact result displays

display_aut = False
display_far = False
display_close = False
display_maxemiss = False
display_minemiss = False

### Autarky Scenario Results

In [37]:
if display_aut == True:
    CHN_aut.total_impacts_phase
    KHM_aut.total_impacts_phase
    LAO_aut.total_impacts_phase
    MMR_aut.total_impacts_phase
    THA_aut.total_impacts_phase
    VNM_aut.total_impacts_phase
    print('---------------------')
    CHN_aut.total_impacts_lifetime
    KHM_aut.total_impacts_lifetime
    LAO_aut.total_impacts_lifetime
    MMR_aut.total_impacts_lifetime
    THA_aut.total_impacts_lifetime
    VNM_aut.total_impacts_lifetime

### Longest Trade Route Scenario Results

In [38]:
if display_far == True:
    CHN_far.total_impacts_phase
    KHM_far.total_impacts_phase
    LAO_far.total_impacts_phase
    MMR_far.total_impacts_phase
    THA_far.total_impacts_phase
    VNM_far.total_impacts_phase
    print('---------------------')
    CHN_far.total_impacts_lifetime
    KHM_far.total_impacts_lifetime
    LAO_far.total_impacts_lifetime
    MMR_far.total_impacts_lifetime
    THA_far.total_impacts_lifetime
    VNM_far.total_impacts_lifetime

### Shortest Trade Route Scenario Results

In [39]:
if display_close == True:
    CHN_close.total_impacts_phase
    KHM_close.total_impacts_phase
    LAO_close.total_impacts_phase
    MMR_close.total_impacts_phase
    THA_close.total_impacts_phase
    VNM_close.total_impacts_phase
    print('---------------------')
    CHN_close.total_impacts_lifetime
    KHM_close.total_impacts_lifetime
    LAO_close.total_impacts_lifetime
    MMR_close.total_impacts_lifetime
    THA_close.total_impacts_lifetime
    VNM_close.total_impacts_lifetime

### Highest-Emission Energy Mix Scenario Results

In [40]:
if display_maxemiss == True:
    CHN_maxemiss.total_impacts_phase
    KHM_maxemiss.total_impacts_phase
    LAO_maxemiss.total_impacts_phase
    MMR_maxemiss.total_impacts_phase
    THA_maxemiss.total_impacts_phase
    VNM_maxemiss.total_impacts_phase
    print('---------------------')
    CHN_maxemiss.total_impacts_lifetime
    KHM_maxemiss.total_impacts_lifetime
    LAO_maxemiss.total_impacts_lifetime
    MMR_maxemiss.total_impacts_lifetime
    THA_maxemiss.total_impacts_lifetime
    VNM_maxemiss.total_impacts_lifetime

### Lowest-Emission Energy Mix Scenario Results

In [41]:
if display_minemiss == True:
    CHN_minemiss.total_impacts_phase
    KHM_minemiss.total_impacts_phase
    LAO_minemiss.total_impacts_phase
    MMR_minemiss.total_impacts_phase
    THA_minemiss.total_impacts_phase
    VNM_minemiss.total_impacts_phase
    print('---------------------')
    CHN_minemiss.total_impacts_lifetime
    KHM_minemiss.total_impacts_lifetime
    LAO_minemiss.total_impacts_lifetime
    MMR_minemiss.total_impacts_lifetime
    THA_minemiss.total_impacts_lifetime
    VNM_minemiss.total_impacts_lifetime

## Concatenate Impact Data

### Autarky Scenario Results

In [42]:
autarky_total_impact_phase = pd.concat([
    CHN_aut.total_impacts_phase,
    KHM_aut.total_impacts_phase,
    LAO_aut.total_impacts_phase,
    MMR_aut.total_impacts_phase,
    THA_aut.total_impacts_phase,
    VNM_aut.total_impacts_phase
])

autarky_total_impact_lifetime = pd.concat([
    CHN_aut.total_impacts_lifetime,
    KHM_aut.total_impacts_lifetime,
    LAO_aut.total_impacts_lifetime,
    MMR_aut.total_impacts_lifetime,
    THA_aut.total_impacts_lifetime,
    VNM_aut.total_impacts_lifetime
])

### Longest Trade Route Scenario Results

In [43]:
far_total_impact_phase = pd.concat([
    CHN_far.total_impacts_phase,
    KHM_far.total_impacts_phase,
    LAO_far.total_impacts_phase,
    MMR_far.total_impacts_phase,
    THA_far.total_impacts_phase,
    VNM_far.total_impacts_phase
])

far_total_impact_lifetime = pd.concat([
    CHN_far.total_impacts_lifetime,
    KHM_far.total_impacts_lifetime,
    LAO_far.total_impacts_lifetime,
    MMR_far.total_impacts_lifetime,
    THA_far.total_impacts_lifetime,
    VNM_far.total_impacts_lifetime
])

### Shortest Trade Route Scenario Results

In [44]:
close_total_impact_phase = pd.concat([
    CHN_close.total_impacts_phase,
    KHM_close.total_impacts_phase,
    LAO_close.total_impacts_phase,
    MMR_close.total_impacts_phase,
    THA_close.total_impacts_phase,
    VNM_close.total_impacts_phase
])

close_total_impact_lifetime = pd.concat([
    CHN_close.total_impacts_lifetime,
    KHM_close.total_impacts_lifetime,
    LAO_close.total_impacts_lifetime,
    MMR_close.total_impacts_lifetime,
    THA_close.total_impacts_lifetime,
    VNM_close.total_impacts_lifetime
])

### Highest-Emission Energy Mix Scenario Results

In [45]:
maxemiss_total_impact_phase = pd.concat([
    CHN_maxemiss.total_impacts_phase,
    KHM_maxemiss.total_impacts_phase,
    LAO_maxemiss.total_impacts_phase,
    MMR_maxemiss.total_impacts_phase,
    THA_maxemiss.total_impacts_phase,
    VNM_maxemiss.total_impacts_phase
])

maxemiss_total_impact_lifetime = pd.concat([
    CHN_maxemiss.total_impacts_lifetime,
    KHM_maxemiss.total_impacts_lifetime,
    LAO_maxemiss.total_impacts_lifetime,
    MMR_maxemiss.total_impacts_lifetime,
    THA_maxemiss.total_impacts_lifetime,
    VNM_maxemiss.total_impacts_lifetime
])

### Lowest-Emission Energy Mix Scenario Results

In [46]:
minemiss_total_impact_phase = pd.concat([
    CHN_minemiss.total_impacts_phase,
    KHM_minemiss.total_impacts_phase,
    LAO_minemiss.total_impacts_phase,
    MMR_minemiss.total_impacts_phase,
    THA_minemiss.total_impacts_phase,
    VNM_minemiss.total_impacts_phase
])

minemiss_total_impact_lifetime = pd.concat([
    CHN_minemiss.total_impacts_lifetime,
    KHM_minemiss.total_impacts_lifetime,
    LAO_minemiss.total_impacts_lifetime,
    MMR_minemiss.total_impacts_lifetime,
    THA_minemiss.total_impacts_lifetime,
    VNM_minemiss.total_impacts_lifetime
])

### Lifetime CO2eq Emissions - Scenario Comparison

In [47]:
# Name of column containing lifetime CO2eq emission data
col_name = 'CO2_eq_kg'

#
CO2_total_impact_lifetime = pd.DataFrame({
    'Autarky': [CHN_aut.total_impacts_lifetime[col_name][0],
                KHM_aut.total_impacts_lifetime[col_name][0],
                LAO_aut.total_impacts_lifetime[col_name][0],
                MMR_aut.total_impacts_lifetime[col_name][0],
                THA_aut.total_impacts_lifetime[col_name][0],
                VNM_aut.total_impacts_lifetime[col_name][0]
               ],
    'Far': [CHN_far.total_impacts_lifetime[col_name][0],
                KHM_far.total_impacts_lifetime[col_name][0],
                LAO_far.total_impacts_lifetime[col_name][0],
                MMR_far.total_impacts_lifetime[col_name][0],
                THA_far.total_impacts_lifetime[col_name][0],
                VNM_far.total_impacts_lifetime[col_name][0]
               ],
    'Close': [CHN_close.total_impacts_lifetime[col_name][0],
                KHM_close.total_impacts_lifetime[col_name][0],
                LAO_close.total_impacts_lifetime[col_name][0],
                MMR_close.total_impacts_lifetime[col_name][0],
                THA_close.total_impacts_lifetime[col_name][0],
                VNM_close.total_impacts_lifetime[col_name][0]
               ],
    'Maxemiss': [CHN_maxemiss.total_impacts_lifetime[col_name][0],
                KHM_maxemiss.total_impacts_lifetime[col_name][0],
                LAO_maxemiss.total_impacts_lifetime[col_name][0],
                MMR_maxemiss.total_impacts_lifetime[col_name][0],
                THA_maxemiss.total_impacts_lifetime[col_name][0],
                VNM_maxemiss.total_impacts_lifetime[col_name][0]
               ],
    'Minemiss': [CHN_minemiss.total_impacts_lifetime[col_name][0],
                KHM_minemiss.total_impacts_lifetime[col_name][0],
                LAO_minemiss.total_impacts_lifetime[col_name][0],
                MMR_minemiss.total_impacts_lifetime[col_name][0],
                THA_minemiss.total_impacts_lifetime[col_name][0],
                VNM_minemiss.total_impacts_lifetime[col_name][0]
               ]},
    index = ['China','Cambodia','Lao PDR', 'Myanmar', 'Thailand', 'Vietnam']
)

### Lifetime SO2eq Emissions - Scenario Comparison

In [48]:
# Name of column containing lifetime SO2eq emission data
col_name = 'SO2_eq_kg'

#
SO2_total_impact_lifetime = pd.DataFrame({
    'Autarky': [CHN_aut.total_impacts_lifetime[col_name][0],
                KHM_aut.total_impacts_lifetime[col_name][0],
                LAO_aut.total_impacts_lifetime[col_name][0],
                MMR_aut.total_impacts_lifetime[col_name][0],
                THA_aut.total_impacts_lifetime[col_name][0],
                VNM_aut.total_impacts_lifetime[col_name][0]
               ],
    'Far': [CHN_far.total_impacts_lifetime[col_name][0],
                KHM_far.total_impacts_lifetime[col_name][0],
                LAO_far.total_impacts_lifetime[col_name][0],
                MMR_far.total_impacts_lifetime[col_name][0],
                THA_far.total_impacts_lifetime[col_name][0],
                VNM_far.total_impacts_lifetime[col_name][0]
               ],
    'Close': [CHN_close.total_impacts_lifetime[col_name][0],
                KHM_close.total_impacts_lifetime[col_name][0],
                LAO_close.total_impacts_lifetime[col_name][0],
                MMR_close.total_impacts_lifetime[col_name][0],
                THA_close.total_impacts_lifetime[col_name][0],
                VNM_close.total_impacts_lifetime[col_name][0]
               ],
    'Maxemiss': [CHN_maxemiss.total_impacts_lifetime[col_name][0],
                KHM_maxemiss.total_impacts_lifetime[col_name][0],
                LAO_maxemiss.total_impacts_lifetime[col_name][0],
                MMR_maxemiss.total_impacts_lifetime[col_name][0],
                THA_maxemiss.total_impacts_lifetime[col_name][0],
                VNM_maxemiss.total_impacts_lifetime[col_name][0]
               ],
    'Minemiss': [CHN_minemiss.total_impacts_lifetime[col_name][0],
                KHM_minemiss.total_impacts_lifetime[col_name][0],
                LAO_minemiss.total_impacts_lifetime[col_name][0],
                MMR_minemiss.total_impacts_lifetime[col_name][0],
                THA_minemiss.total_impacts_lifetime[col_name][0],
                VNM_minemiss.total_impacts_lifetime[col_name][0]
               ]},
    index = ['China','Cambodia','Lao PDR', 'Myanmar', 'Thailand', 'Vietnam']
)

### Lifetime PM2.5eq Emissions - Scenario Comparison

In [49]:
# Name of column containing lifetime PM25eq emission data
col_name = 'PM25_eq_kg'

#
PM25_total_impact_lifetime = pd.DataFrame({
    'Autarky': [CHN_aut.total_impacts_lifetime[col_name][0],
                KHM_aut.total_impacts_lifetime[col_name][0],
                LAO_aut.total_impacts_lifetime[col_name][0],
                MMR_aut.total_impacts_lifetime[col_name][0],
                THA_aut.total_impacts_lifetime[col_name][0],
                VNM_aut.total_impacts_lifetime[col_name][0]
               ],
    'Far': [CHN_far.total_impacts_lifetime[col_name][0],
                KHM_far.total_impacts_lifetime[col_name][0],
                LAO_far.total_impacts_lifetime[col_name][0],
                MMR_far.total_impacts_lifetime[col_name][0],
                THA_far.total_impacts_lifetime[col_name][0],
                VNM_far.total_impacts_lifetime[col_name][0]
               ],
    'Close': [CHN_close.total_impacts_lifetime[col_name][0],
                KHM_close.total_impacts_lifetime[col_name][0],
                LAO_close.total_impacts_lifetime[col_name][0],
                MMR_close.total_impacts_lifetime[col_name][0],
                THA_close.total_impacts_lifetime[col_name][0],
                VNM_close.total_impacts_lifetime[col_name][0]
               ],
    'Maxemiss': [CHN_maxemiss.total_impacts_lifetime[col_name][0],
                KHM_maxemiss.total_impacts_lifetime[col_name][0],
                LAO_maxemiss.total_impacts_lifetime[col_name][0],
                MMR_maxemiss.total_impacts_lifetime[col_name][0],
                THA_maxemiss.total_impacts_lifetime[col_name][0],
                VNM_maxemiss.total_impacts_lifetime[col_name][0]
               ],
    'Minemiss': [CHN_minemiss.total_impacts_lifetime[col_name][0],
                KHM_minemiss.total_impacts_lifetime[col_name][0],
                LAO_minemiss.total_impacts_lifetime[col_name][0],
                MMR_minemiss.total_impacts_lifetime[col_name][0],
                THA_minemiss.total_impacts_lifetime[col_name][0],
                VNM_minemiss.total_impacts_lifetime[col_name][0]
               ]},
    index = ['China','Cambodia','Lao PDR', 'Myanmar', 'Thailand', 'Vietnam']
)

### Material Extraction Phase CO2eq Emissions - Scenario Comparison

In [50]:
# Name of column containing lifetime CO2eq emission data
col_name = 'CO2_eq_kg'

# Name of phase of interest; options are 'materials_extraction', 'construction', 'operation'
phase = 'materials_extraction'

#
CO2_phase_impact_materials_extraction = pd.DataFrame({
    'Autarky': [CHN_aut.total_impacts_phase[CHN_aut.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                KHM_aut.total_impacts_phase[KHM_aut.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                LAO_aut.total_impacts_phase[LAO_aut.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                MMR_aut.total_impacts_phase[MMR_aut.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                THA_aut.total_impacts_phase[THA_aut.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                VNM_aut.total_impacts_phase[VNM_aut.total_impacts_phase.index.str.startswith(phase)][col_name][0]
               ],
    'Far': [CHN_far.total_impacts_phase[CHN_far.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                KHM_far.total_impacts_phase[KHM_far.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                LAO_far.total_impacts_phase[LAO_far.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                MMR_far.total_impacts_phase[MMR_far.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                THA_far.total_impacts_phase[THA_far.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                VNM_far.total_impacts_phase[VNM_far.total_impacts_phase.index.str.startswith(phase)][col_name][0]
               ],
    'Close': [CHN_close.total_impacts_phase[CHN_close.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                KHM_close.total_impacts_phase[KHM_close.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                LAO_close.total_impacts_phase[LAO_close.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                MMR_close.total_impacts_phase[MMR_close.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                THA_close.total_impacts_phase[THA_close.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                VNM_close.total_impacts_phase[VNM_close.total_impacts_phase.index.str.startswith(phase)][col_name][0]
               ],
    'Maxemiss': [CHN_maxemiss.total_impacts_phase[CHN_maxemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                KHM_maxemiss.total_impacts_phase[KHM_maxemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                LAO_maxemiss.total_impacts_phase[LAO_maxemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                MMR_maxemiss.total_impacts_phase[MMR_maxemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                THA_maxemiss.total_impacts_phase[THA_maxemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                VNM_maxemiss.total_impacts_phase[VNM_maxemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0]
               ],
    'Minemiss': [CHN_minemiss.total_impacts_phase[CHN_minemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                KHM_minemiss.total_impacts_phase[KHM_minemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                LAO_minemiss.total_impacts_phase[LAO_minemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                MMR_minemiss.total_impacts_phase[MMR_minemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                THA_minemiss.total_impacts_phase[THA_minemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                VNM_minemiss.total_impacts_phase[VNM_minemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0]
               ]},
    index = ['China','Cambodia','Lao PDR', 'Myanmar', 'Thailand', 'Vietnam']
)

### Material Extraction Phase SO2eq Emissions - Scenario Comparison

In [51]:
# Name of column containing lifetime SO2eq emission data
col_name = 'SO2_eq_kg'

# Name of phase of interest; options are 'materials_extraction', 'construction', 'operation'
phase = 'materials_extraction'

#
SO2_phase_impact_materials_extraction = pd.DataFrame({
    'Autarky': [CHN_aut.total_impacts_phase[CHN_aut.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                KHM_aut.total_impacts_phase[KHM_aut.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                LAO_aut.total_impacts_phase[LAO_aut.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                MMR_aut.total_impacts_phase[MMR_aut.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                THA_aut.total_impacts_phase[THA_aut.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                VNM_aut.total_impacts_phase[VNM_aut.total_impacts_phase.index.str.startswith(phase)][col_name][0]
               ],
    'Far': [CHN_far.total_impacts_phase[CHN_far.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                KHM_far.total_impacts_phase[KHM_far.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                LAO_far.total_impacts_phase[LAO_far.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                MMR_far.total_impacts_phase[MMR_far.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                THA_far.total_impacts_phase[THA_far.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                VNM_far.total_impacts_phase[VNM_far.total_impacts_phase.index.str.startswith(phase)][col_name][0]
               ],
    'Close': [CHN_close.total_impacts_phase[CHN_close.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                KHM_close.total_impacts_phase[KHM_close.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                LAO_close.total_impacts_phase[LAO_close.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                MMR_close.total_impacts_phase[MMR_close.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                THA_close.total_impacts_phase[THA_close.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                VNM_close.total_impacts_phase[VNM_close.total_impacts_phase.index.str.startswith(phase)][col_name][0]
               ],
    'Maxemiss': [CHN_maxemiss.total_impacts_phase[CHN_maxemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                KHM_maxemiss.total_impacts_phase[KHM_maxemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                LAO_maxemiss.total_impacts_phase[LAO_maxemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                MMR_maxemiss.total_impacts_phase[MMR_maxemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                THA_maxemiss.total_impacts_phase[THA_maxemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                VNM_maxemiss.total_impacts_phase[VNM_maxemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0]
               ],
    'Minemiss': [CHN_minemiss.total_impacts_phase[CHN_minemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                KHM_minemiss.total_impacts_phase[KHM_minemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                LAO_minemiss.total_impacts_phase[LAO_minemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                MMR_minemiss.total_impacts_phase[MMR_minemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                THA_minemiss.total_impacts_phase[THA_minemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                VNM_minemiss.total_impacts_phase[VNM_minemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0]
               ]},
    index = ['China','Cambodia','Lao PDR', 'Myanmar', 'Thailand', 'Vietnam']
)

### Material Extraction Phase PM2.5eq Emissions - Scenario Comparison

In [52]:
# Name of column containing lifetime PM25eq emission data
col_name = 'PM25_eq_kg'

# Name of phase of interest; options are 'materials_extraction', 'construction', 'operation'
phase = 'materials_extraction'

#
PM25_phase_impact_materials_extraction = pd.DataFrame({
    'Autarky': [CHN_aut.total_impacts_phase[CHN_aut.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                KHM_aut.total_impacts_phase[KHM_aut.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                LAO_aut.total_impacts_phase[LAO_aut.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                MMR_aut.total_impacts_phase[MMR_aut.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                THA_aut.total_impacts_phase[THA_aut.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                VNM_aut.total_impacts_phase[VNM_aut.total_impacts_phase.index.str.startswith(phase)][col_name][0]
               ],
    'Far': [CHN_far.total_impacts_phase[CHN_far.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                KHM_far.total_impacts_phase[KHM_far.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                LAO_far.total_impacts_phase[LAO_far.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                MMR_far.total_impacts_phase[MMR_far.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                THA_far.total_impacts_phase[THA_far.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                VNM_far.total_impacts_phase[VNM_far.total_impacts_phase.index.str.startswith(phase)][col_name][0]
               ],
    'Close': [CHN_close.total_impacts_phase[CHN_close.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                KHM_close.total_impacts_phase[KHM_close.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                LAO_close.total_impacts_phase[LAO_close.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                MMR_close.total_impacts_phase[MMR_close.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                THA_close.total_impacts_phase[THA_close.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                VNM_close.total_impacts_phase[VNM_close.total_impacts_phase.index.str.startswith(phase)][col_name][0]
               ],
    'Maxemiss': [CHN_maxemiss.total_impacts_phase[CHN_maxemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                KHM_maxemiss.total_impacts_phase[KHM_maxemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                LAO_maxemiss.total_impacts_phase[LAO_maxemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                MMR_maxemiss.total_impacts_phase[MMR_maxemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                THA_maxemiss.total_impacts_phase[THA_maxemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                VNM_maxemiss.total_impacts_phase[VNM_maxemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0]
               ],
    'Minemiss': [CHN_minemiss.total_impacts_phase[CHN_minemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                KHM_minemiss.total_impacts_phase[KHM_minemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                LAO_minemiss.total_impacts_phase[LAO_minemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                MMR_minemiss.total_impacts_phase[MMR_minemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                THA_minemiss.total_impacts_phase[THA_minemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                VNM_minemiss.total_impacts_phase[VNM_minemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0]
               ]},
    index = ['China','Cambodia','Lao PDR', 'Myanmar', 'Thailand', 'Vietnam']
)

### Construction Phase CO2eq Emissions - Scenario Comparison

In [53]:
# Name of column containing lifetime CO2eq emission data
col_name = 'CO2_eq_kg'

# Name of phase of interest; options are 'materials_extraction', 'construction', 'operation'
phase = 'construction'

#
CO2_phase_impact_construction = pd.DataFrame({
    'Autarky': [CHN_aut.total_impacts_phase[CHN_aut.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                KHM_aut.total_impacts_phase[KHM_aut.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                LAO_aut.total_impacts_phase[LAO_aut.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                MMR_aut.total_impacts_phase[MMR_aut.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                THA_aut.total_impacts_phase[THA_aut.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                VNM_aut.total_impacts_phase[VNM_aut.total_impacts_phase.index.str.startswith(phase)][col_name][0]
               ],
    'Far': [CHN_far.total_impacts_phase[CHN_far.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                KHM_far.total_impacts_phase[KHM_far.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                LAO_far.total_impacts_phase[LAO_far.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                MMR_far.total_impacts_phase[MMR_far.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                THA_far.total_impacts_phase[THA_far.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                VNM_far.total_impacts_phase[VNM_far.total_impacts_phase.index.str.startswith(phase)][col_name][0]
               ],
    'Close': [CHN_close.total_impacts_phase[CHN_close.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                KHM_close.total_impacts_phase[KHM_close.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                LAO_close.total_impacts_phase[LAO_close.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                MMR_close.total_impacts_phase[MMR_close.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                THA_close.total_impacts_phase[THA_close.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                VNM_close.total_impacts_phase[VNM_close.total_impacts_phase.index.str.startswith(phase)][col_name][0]
               ],
    'Maxemiss': [CHN_maxemiss.total_impacts_phase[CHN_maxemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                KHM_maxemiss.total_impacts_phase[KHM_maxemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                LAO_maxemiss.total_impacts_phase[LAO_maxemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                MMR_maxemiss.total_impacts_phase[MMR_maxemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                THA_maxemiss.total_impacts_phase[THA_maxemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                VNM_maxemiss.total_impacts_phase[VNM_maxemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0]
               ],
    'Minemiss': [CHN_minemiss.total_impacts_phase[CHN_minemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                KHM_minemiss.total_impacts_phase[KHM_minemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                LAO_minemiss.total_impacts_phase[LAO_minemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                MMR_minemiss.total_impacts_phase[MMR_minemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                THA_minemiss.total_impacts_phase[THA_minemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                VNM_minemiss.total_impacts_phase[VNM_minemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0]
               ]},
    index = ['China','Cambodia','Lao PDR', 'Myanmar', 'Thailand', 'Vietnam']
)

### Construction Phase SO2eq Emissions - Scenario Comparison

In [54]:
# Name of column containing lifetime SO2eq emission data
col_name = 'SO2_eq_kg'

# Name of phase of interest; options are 'materials_extraction', 'construction', 'operation'
phase = 'construction'

#
SO2_phase_impact_construction = pd.DataFrame({
    'Autarky': [CHN_aut.total_impacts_phase[CHN_aut.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                KHM_aut.total_impacts_phase[KHM_aut.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                LAO_aut.total_impacts_phase[LAO_aut.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                MMR_aut.total_impacts_phase[MMR_aut.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                THA_aut.total_impacts_phase[THA_aut.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                VNM_aut.total_impacts_phase[VNM_aut.total_impacts_phase.index.str.startswith(phase)][col_name][0]
               ],
    'Far': [CHN_far.total_impacts_phase[CHN_far.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                KHM_far.total_impacts_phase[KHM_far.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                LAO_far.total_impacts_phase[LAO_far.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                MMR_far.total_impacts_phase[MMR_far.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                THA_far.total_impacts_phase[THA_far.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                VNM_far.total_impacts_phase[VNM_far.total_impacts_phase.index.str.startswith(phase)][col_name][0]
               ],
    'Close': [CHN_close.total_impacts_phase[CHN_close.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                KHM_close.total_impacts_phase[KHM_close.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                LAO_close.total_impacts_phase[LAO_close.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                MMR_close.total_impacts_phase[MMR_close.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                THA_close.total_impacts_phase[THA_close.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                VNM_close.total_impacts_phase[VNM_close.total_impacts_phase.index.str.startswith(phase)][col_name][0]
               ],
    'Maxemiss': [CHN_maxemiss.total_impacts_phase[CHN_maxemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                KHM_maxemiss.total_impacts_phase[KHM_maxemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                LAO_maxemiss.total_impacts_phase[LAO_maxemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                MMR_maxemiss.total_impacts_phase[MMR_maxemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                THA_maxemiss.total_impacts_phase[THA_maxemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                VNM_maxemiss.total_impacts_phase[VNM_maxemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0]
               ],
    'Minemiss': [CHN_minemiss.total_impacts_phase[CHN_minemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                KHM_minemiss.total_impacts_phase[KHM_minemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                LAO_minemiss.total_impacts_phase[LAO_minemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                MMR_minemiss.total_impacts_phase[MMR_minemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                THA_minemiss.total_impacts_phase[THA_minemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                VNM_minemiss.total_impacts_phase[VNM_minemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0]
               ]},
    index = ['China','Cambodia','Lao PDR', 'Myanmar', 'Thailand', 'Vietnam']
)

### Construction Phase PM2.5eq Emissions - Scenario Comparison

In [55]:
# Name of column containing lifetime PM25eq emission data
col_name = 'PM25_eq_kg'

# Name of phase of interest; options are 'materials_extraction', 'construction', 'operation'
phase = 'construction'

#
PM25_phase_impact_construction = pd.DataFrame({
    'Autarky': [CHN_aut.total_impacts_phase[CHN_aut.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                KHM_aut.total_impacts_phase[KHM_aut.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                LAO_aut.total_impacts_phase[LAO_aut.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                MMR_aut.total_impacts_phase[MMR_aut.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                THA_aut.total_impacts_phase[THA_aut.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                VNM_aut.total_impacts_phase[VNM_aut.total_impacts_phase.index.str.startswith(phase)][col_name][0]
               ],
    'Far': [CHN_far.total_impacts_phase[CHN_far.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                KHM_far.total_impacts_phase[KHM_far.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                LAO_far.total_impacts_phase[LAO_far.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                MMR_far.total_impacts_phase[MMR_far.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                THA_far.total_impacts_phase[THA_far.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                VNM_far.total_impacts_phase[VNM_far.total_impacts_phase.index.str.startswith(phase)][col_name][0]
               ],
    'Close': [CHN_close.total_impacts_phase[CHN_close.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                KHM_close.total_impacts_phase[KHM_close.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                LAO_close.total_impacts_phase[LAO_close.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                MMR_close.total_impacts_phase[MMR_close.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                THA_close.total_impacts_phase[THA_close.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                VNM_close.total_impacts_phase[VNM_close.total_impacts_phase.index.str.startswith(phase)][col_name][0]
               ],
    'Maxemiss': [CHN_maxemiss.total_impacts_phase[CHN_maxemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                KHM_maxemiss.total_impacts_phase[KHM_maxemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                LAO_maxemiss.total_impacts_phase[LAO_maxemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                MMR_maxemiss.total_impacts_phase[MMR_maxemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                THA_maxemiss.total_impacts_phase[THA_maxemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                VNM_maxemiss.total_impacts_phase[VNM_maxemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0]
               ],
    'Minemiss': [CHN_minemiss.total_impacts_phase[CHN_minemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                KHM_minemiss.total_impacts_phase[KHM_minemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                LAO_minemiss.total_impacts_phase[LAO_minemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                MMR_minemiss.total_impacts_phase[MMR_minemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                THA_minemiss.total_impacts_phase[THA_minemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0],
                VNM_minemiss.total_impacts_phase[VNM_minemiss.total_impacts_phase.index.str.startswith(phase)][col_name][0]
               ]},
    index = ['China','Cambodia','Lao PDR', 'Myanmar', 'Thailand', 'Vietnam']
)

### Energy Mix Unit Impacts

In [56]:
CO2eq_list = ['CO2_kg','N20_kg','CH4_kg','CFCs_kg','HCFCs_kg','CH3Br_kg']
SO2eq_list = ['SO2_kg','SO3_kg','HCl_kg','HF_kg','NOx_kg','NH4_kg']
PM25eq_list = ['PM25_kg', 'PM25-10_kg', 'PM10_kg']

CO2eq = CHN_aut.energy_mix_emissions[CO2eq_list].sum(axis=1)
SO2eq = CHN_aut.energy_mix_emissions[SO2eq_list].sum(axis=1)
PM25eq = CHN_aut.energy_mix_emissions[PM25eq_list].sum(axis=1)

# Note that the energy_mix_emissions table is identical for all class instances;
# only one copy need be exported
energy_mix_impacts = pd.DataFrame({'CO2eq': CHN_aut.energy_mix_emissions[CO2eq_list].sum(axis=1),
                                   'SO2eq': CHN_aut.energy_mix_emissions[SO2eq_list].sum(axis=1),
                                   'PM25eq': CHN_aut.energy_mix_emissions[PM25eq_list].sum(axis=1)}
                                 )
energy_mix_impacts.index = ['China','Cambodia','Lao PDR', 'Myanmar', 'Thailand', 'Vietnam']

## Export Data

In [57]:
# If selected results export directory does not exist, create it
if not os.path.exists(export_dir):
    os.makedirs(export_dir)

### Total Impact - Phase - CSV Exports

In [58]:
autarky_total_impact_phase.to_csv(
    export_dir + "/autarky_total_impact_phase.csv")
far_total_impact_phase.to_csv(
    export_dir + "/far_total_impact_phase.csv")
close_total_impact_phase.to_csv(
    export_dir + "/close_total_impact_phase.csv")
maxemiss_total_impact_phase.to_csv(
    export_dir + "/maxemiss_total_impact_phase.csv")
minemiss_total_impact_phase.to_csv(
    export_dir + "/minemiss_total_impact_phase.csv")

### Total Impact - Lifetime - CSV Exports

In [59]:
autarky_total_impact_lifetime.to_csv(
    export_dir + "/autarky_total_impact_lifetime.csv")
far_total_impact_lifetime.to_csv(
    export_dir + "/far_total_impact_lifetime.csv")
close_total_impact_lifetime.to_csv(
    export_dir + "/close_total_impact_lifetime.csv")
maxemiss_total_impact_lifetime.to_csv(
    export_dir + "/maxemiss_total_impact_lifetime.csv")
minemiss_total_impact_lifetime.to_csv(
    export_dir + "/minemiss_total_impact_lifetime.csv")

### Total Impact - Phase - Excel Export

In [60]:
with pd.ExcelWriter(export_dir + "/total_impact_phase_all.xlsx") as writer:
    autarky_total_impact_phase.to_excel(writer, sheet_name='autarky_total_impact_phase')
    far_total_impact_phase.to_excel(writer, sheet_name='far_total_impact_phase')
    close_total_impact_phase.to_excel(writer, sheet_name='close_total_impact_phase')
    maxemiss_total_impact_phase.to_excel(writer, sheet_name='maxemiss_total_impact_phase')
    minemiss_total_impact_phase.to_excel(writer, sheet_name='minemiss_total_impact_phase')

### Total Impact - Lifetime - Excel Export

In [61]:
with pd.ExcelWriter(export_dir + "/total_impact_lifetime_all.xlsx") as writer:
    autarky_total_impact_lifetime.to_excel(writer, sheet_name='autarky_total_impact_lifetime')
    far_total_impact_lifetime.to_excel(writer, sheet_name='far_total_impact_lifetime')
    close_total_impact_lifetime.to_excel(writer, sheet_name='close_total_impact_lifetime')
    maxemiss_total_impact_lifetime.to_excel(writer, sheet_name='maxemiss_total_impact_lifetime')
    minemiss_total_impact_lifetime.to_excel(writer, sheet_name='minemiss_total_impact_lifetime')

### Unit Energy Mix Emissions - Excel Export

In [62]:
# Note that the energy_mix_emissions table is identical for all class instances;
# only one copy need be exported

with pd.ExcelWriter(export_dir + "/energy_mix_emission.xlsx") as writer:
    CHN_aut.energy_mix_emissions.to_excel(writer, sheet_name='emissions_table')

### Results Summary - Excel Export

In [63]:
with pd.ExcelWriter(export_dir + "/results_summary.xlsx") as writer:
    CO2_total_impact_lifetime.to_excel(writer, sheet_name='lifetime_impacts_CO2')
    SO2_total_impact_lifetime.to_excel(writer, sheet_name='lifetime_impacts_SO2')
    PM25_total_impact_lifetime.to_excel(writer, sheet_name='lifetime_impacts_PM25')
    CO2_phase_impact_materials_extraction.to_excel(writer, sheet_name='matextract_impacts_CO2')
    SO2_phase_impact_materials_extraction.to_excel(writer, sheet_name='matextract_impacts_SO2')
    PM25_phase_impact_materials_extraction.to_excel(writer, sheet_name='matextract_impacts_PM25')
    CO2_phase_impact_construction.to_excel(writer, sheet_name='construction_impacts_CO2')
    SO2_phase_impact_construction.to_excel(writer, sheet_name='construction_impacts_SO2')
    PM25_phase_impact_construction.to_excel(writer, sheet_name='construction_impacts_PM25')
    energy_mix_impacts.to_excel(writer, sheet_name='energy_mix_unit_impacts')